In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
from datetime import date, timedelta

root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.ingestion import volume_acucar_exportacao
from src.features.data import criar_tabela_spark, salvar_tabela_delta_spark
from src.features.processing import renomear_colunas, data_ingestao

In [0]:
# construção da tabela histórica de volume de açúcar não refinado exportado
TABELA_INCREMENTAL_RAW = "`api-forecasting`.bronze.volume_exportacao_acucar_incremental_raw"

data_atual = date.today().strftime('%Y-%m')
try:
    volume_exportacao = volume_acucar_exportacao(mes_ano_inicio='2025-10', mes_ano_fim='2025-10')

except Exception as e:
    raise print('Dado ainda não disponível para captura')


In [0]:
# inserção da coluna de data de ingestao
volume_exportacao = data_ingestao(volume_exportacao)
volume_exportacao.head()


In [0]:
# transformação do dataset no formato spark
df_spark = criar_tabela_spark(volume_exportacao)

In [0]:
# visualização dos dados
df_spark.show()

In [0]:
# persistência da tabela na camada bronze
df_spark = salvar_tabela_delta_spark(df_spark, 
                         mode='append', partitionby=['date'],merge_schema=True, path=TABELA_INCREMENTAL_RAW)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.volume_exportacao_acucar_incremental_raw
LIMIT 10

In [0]:
%sql
drop table if exists `api-forecasting`.bronze.precos_cepea_historico_raw